<a href="https://colab.research.google.com/github/ceciliasecchi/MasterThesis/blob/main/MasterThesis3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wordfreq
from wordfreq import word_frequency
from nltk.collections import OrderedDict
import gensim
import pandas as pd
import nltk
nltk.download('punkt')
import re  # For preprocessing
import string
import numpy as np
from numpy import linalg as LA
import csv
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
from gensim.parsing.preprocessing import remove_stopwords,preprocess_string,preprocess_documents
warnings.filterwarnings(action = 'ignore')
import gensim
from gensim.models import Word2Vec
import collections
import math
import random
from collections import defaultdict
from typing import DefaultDict
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.stats import bernoulli 
from scipy.special import softmax as scipy_softmax


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 56.8 MB 18 kB/s 
     |████████████████████████████████| 53 kB 1.2 MB/s 


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
# Reads ‘alice.txt’ file
sample = open("alice_firstpage.txt", "r")
s = sample.read()
stopp = {'and','but','this','said','you','the','how','for','like','thats',
         'like','thought','again','but','dont','that','ive','didnt','they','she',
         'while','instead','would','thats','anything','its','hasnt','still',
         'gets','went',"wouldnt","get","let",'way',"these","those","havent"
         'looked','came','not','got','then','ive',"were",'there','wont'}
voc = {}
vocGlove = OrderedDict()
vocGloveReverse = {}
with open('vocGlove_Alice.csv', mode='r') as inp:
    reader = csv.reader(inp)
    for row in reader:
      voc = {rows[0]:rows[1].replace("\n", " ").replace("["," ").replace("]"," ") for rows in reader}
    
for key,value in voc.items():
  value=value.split()
  if len(value)!=50:
    print(key)
  if key not in stopp:
    vocGlove[key]=np.array(list(map(float,value)))

# normalize the vectors
for k,v in vocGlove.items():
  v = v/np.linalg.norm(v)
  vocGlove[k] = v
  vocGloveReverse[str(v)]=k


In [3]:
dizionario=defaultdict(int)
# Replaces escape character with space
g = s.replace("\n"," ")
g = g.replace("’"," ")
g = g.replace("“", ".")
g = g.replace("”",".")
g = g.replace(";",".")

f = g.split(".")
#re.split('.| ;| “| ”',g)
data = []
# iterate through each sentence in the file
frasi=[]
for i in f:
  copia=i
  i = remove_stopwords(i)
  i = i.translate(i.maketrans('', '', string.punctuation+"\—\-\"\'\“\”\’\‘"))
  temp = []
	
	# tokenize the sentence into words
  for j in word_tokenize(i):
    j = j.lower()
    if j in vocGlove.keys() and len(j)>2:
      dizionario[j] += 1
      temp.append(j)
    else: pass
  if len(temp)>5:
    data.append(temp)
    frasi.append(copia)

def generate_dictionary_data(text):
    word_to_index= OrderedDict()
    index_to_word = OrderedDict()
    freq_dict = DefaultDict(int)
    corpus = []
    corpus_sent = []
    count = 0
    vocab_size = 0
    
    for row in text:
        for word in row:
            freq_dict[word] += 1
            word = word.lower()
            corpus.append(word)
            if word_to_index.get(word) == None:
                word_to_index.update ( {word : count})
                index_to_word.update ( {count : word })
                count  += 1
    vocab_size = len(word_to_index)
    length_of_corpus = len(corpus)
    
    return word_to_index,index_to_word,corpus,vocab_size,length_of_corpus,freq_dict

In [4]:
word_to_index,index_to_word,corpus,vocab_size,length_of_corpus,freq_dict=generate_dictionary_data(data)

In [9]:
sentencesAsVec=[]
num_sent=len(data)

count=0
conteggio={}
for i in range(num_sent):
  len_sent=len(data[i])
  vettori=[]
  for parola in corpus[count:count+len_sent]:
    vettori.append(vocGlove[parola])
  conteggio[i]=[count,count+len_sent]
  count += len_sent
  sentencesAsVec.append(vettori)


# Kernel Distance

In [5]:
K=3 #number of Dirac masses in the compressed distribution
N=50 #ambient dimension
J=1000 #how many samples are included: more the better, otherwise there are oscillations in the SGD/Adam convergence
quant=tf.Variable(np.random.rand(N,K),dtype=tf.float32) #this will be the set of quantization variables


In [6]:
def empirical_sampling_from_sentence(sentence, S=30):
    ''' sample from the sentence (that is a vector of words of length 50) 30 words
        note: sentence is a vector of vectors, then the output is a np.array 
              where each coloumn corresponds to a word
    '''
    N=len(sentence)
    Y=[]
    if N>=S:
        indices=np.random.choice(N, S)
    else:
        indices=np.concatenate((np.arange(N),np.random.choice(N,S-N)),axis=0)
        #Y=sentence
    for i in indices:
        Y.append(sentence[i])
    Y=np.transpose(np.asarray(Y))
    return Y
    #return Y,indices

empirical_sampling=empirical_sampling_from_sentence

def empirical_radon_sobolev_distance_sq(X,Y,local_alphas=None,betas=None):
    '''
    Parameters
    ----------
    X : 2D NxK matrix
        input data sample, each column a vector of dimension N, notation X_k
    Y : same as X for the second distribution
    alphas : 1D array of weights for X, in my case alphas==1/K
    betas : same as alphas for Y , in my case betas==1/30?

    Note: X,alphas are Tensor/Variable while Y/betas are numpy array

    Returns
    -------
    Radon-Sobolev distance
    
    '''
    N,K=X.shape
    Ny,J=Y.shape
    Ytensor = tf.convert_to_tensor(Y, dtype=tf.float32)

    # this is my case.. maybe in the future put tf-idf as weights?
    if local_alphas is None:
      alphas = tf.convert_to_tensor(np.ones(K)/K,dtype=tf.float32)
    else:
      alphas=tf.nn.softmax(local_alphas)
    if betas is None:
        betas = np.ones(J)/J
    betastensor = tf.convert_to_tensor(betas, dtype=tf.float32)

    # serie di check per vedere che gli input abbiano senso in dimensione
    # tf.rank = # indeces required to uniquely select an element
    assert (tf.rank(X).numpy()==2) & (Y.ndim==2) & (tf.rank(alphas).numpy()==1) & (betas.ndim==1),"invalid input dimensions"
    Ka,=alphas.shape
    Jb,=betas.shape
    assert (N==Ny) & (K==Ka) & (J==Jb), 'invalid input dimensions'

    #construct big matrix: [X,Y]=[X_1,...,X_3,Y_1,...,Y_30]
    points=tf.concat([X,Ytensor],axis=1)
    # construct big vector of weights 
    gammas=tf.concat([alphas,-betastensor],axis=0)

    #construct the distances matrix as a K+J x K+J matrix
    distZZ = tf.math.sqrt(1.0e-10+tf.math.reduce_sum(tf.square(tf.expand_dims(points,2)-tf.expand_dims(points,1)),axis=0))-1.0e-5
    # Given a tensor input, this operation returns a tensor of the same type with all dimensions of size 1 removed.
    # I return thee matrix multiplied by the vectors
    return tf.squeeze(-0.5*tf.expand_dims(gammas,0)@distZZ@tf.expand_dims(gammas,1))

def empirical_radon_sobolev_distance_sq_geod(X,Y,local_alphas=None,betas=None):
    '''
    Parameters
    ----------
    X : 2D NxK matrix
        input data sample, each column a vector of dimension N, notation X_k
    Y : same as X for the second distribution
    alphas : 1D array of weights for X, in my case alphas==1/K
    betas : same as alphas for Y , in my case betas==1/30?

    Note: X,alphas are Tensor/Variable while Y/betas are numpy array

    Returns
    -------
    Radon-Sobolev distance
    
    '''
    N,K=X.shape
    Ny,J=Y.shape
    Ytensor = tf.convert_to_tensor(Y, dtype=tf.float32)

    # rinormalizzo ogni volta
    norma=tf.norm(X,axis=0)
    X=X/tf.sqrt(norma**2+1.e-6)


    # this is my case.. maybe in the future put tf-idf as weights?
    if local_alphas is None:
      alphas = tf.convert_to_tensor(np.ones(K)/K,dtype=tf.float32)
    else:
      alphas=tf.nn.softmax(local_alphas)
    if betas is None:
        betas = np.ones(J)/J
    betastensor = tf.convert_to_tensor(betas, dtype=tf.float32)

    # serie di check per vedere che gli input abbiano senso in dimensione
    # tf.rank = # indeces required to uniquely select an element
    assert (tf.rank(X).numpy()==2) & (Y.ndim==2) & (tf.rank(alphas).numpy()==1) & (betas.ndim==1),"invalid input dimensions"
    Ka,=alphas.shape
    Jb,=betas.shape
    assert (N==Ny)& (K==Ka)&(J==Jb), 'invalid input dimensions'


    #construct big matrix: [X,Y]=[X_1,...,X_3,Y_1,...,Y_30]
    #points=np.transpose(tf.concat([X,Ytensor],axis=1))
    points=tf.concat([X,Ytensor],axis=1)

    # construct big vector of weights 
    gammas=tf.concat([alphas,-betastensor],axis=0)

    #construct the distances matrix as a K+J x K+J matrix
    
    distZZ = tf.math.acos(tf.matmul(tf.transpose(points),points)/1.0001)
    # Given a tensor input, this operation returns a tensor of the same type with all dimensions of size 1 removed.
    # I return thee matrix multiplied by the vectors
    return tf.squeeze(-0.5*tf.expand_dims(gammas,0)@distZZ@tf.expand_dims(gammas,1))#-sum(np.multiply(Lag,np.power(np.linalg.norm(X,axis=0),2)))+K

                            

In [7]:
def distanzaH(sentence,compressed):
        J=len(sentence)

        sentence=np.transpose(sentence)
        N,K=compressed.shape
        Ytensor = tf.convert_to_tensor(sentence, dtype=tf.float32)
        alphas = tf.convert_to_tensor(np.ones(K)/K,dtype=tf.float32)
        betas = np.ones(J)/J
        betastensor = tf.convert_to_tensor(betas, dtype=tf.float32)
        points=tf.concat([compressed,sentence],axis=1)
        gammas=tf.concat([alphas,-betastensor],axis=0)
        distZZ = tf.math.sqrt(1.0e-10+tf.math.reduce_sum(tf.square(tf.expand_dims(points,2)-tf.expand_dims(points,1)),axis=0))-1.0e-5
        return tf.squeeze(-0.5*tf.expand_dims(gammas,0)@distZZ@tf.expand_dims(gammas,1))

def distanzaGeo(sentence,compressed):
        J=len(sentence)
        norma=tf.norm(compressed,axis=0)
        compressed=compressed/tf.sqrt(norma**2+1.e-6)
        sentence=np.transpose(sentence)
        N,K=compressed.shape
        Ytensor = tf.convert_to_tensor(sentence, dtype=tf.float32)
        alphas = tf.convert_to_tensor(np.ones(K)/K,dtype=tf.float32)
        betas = np.ones(J)/J
        betastensor = tf.convert_to_tensor(betas, dtype=tf.float32)
        points=tf.concat([compressed,sentence],axis=1)
        gammas=tf.concat([alphas,-betastensor],axis=0)
        distZZ = tf.math.acos(tf.matmul(tf.transpose(points),points)/1.0001)
        return tf.squeeze(-0.5*tf.expand_dims(gammas,0)@distZZ@tf.expand_dims(gammas,1))



## Find one word at a time 
Using as kernel:
$h(x,y)=\sqrt{a^2+|x-y|^2}-a$

In [11]:
  num_sent=len(data)
  step=6
  for w in range(num_sent):
    sentence=sentencesAsVec[w]
    K=3 #number of Dirac masses in the compressed distribution
    N=50 #ambient dimension
    J=1000 #how many samples are included: more the better, otherwise there are oscillations in the SGD/Adam convergence
    quant=tf.Variable(np.random.rand(N,K),dtype=tf.float32) #this will be the set of quantization variables
    opt = tf.keras.optimizers.Adam(learning_rate=0.1,beta_1=0.9, beta_2=0.999)# TODO: use lr_schedule for step decay schedule !
    loss_quant = lambda: empirical_radon_sobolev_distance_sq(quant,empirical_sampling(sentence))

    nr_iter=100
    loss_val=[]
    opt_val=[]

    for sc in range(nr_iter):
      step_count = opt.minimize(loss_quant, [quant]).numpy()
      opt_val.append(quant.numpy())
      loss_val.append(loss_quant())
    c1=quant[:,0]
    c2=quant[:,1]
    c3=quant[:,2]
    compression=[c1,c2,c3]
    cdcd=tf.Variable(np.array(compression).reshape(50,-1), dtype=tf.float32)
    dc=math.sqrt(distanzaH(sentence,cdcd).numpy())
    print(frasi[w])
    N=len(sentence)
    #indici=random.sample(range(N), 3)
    score1={}
    for parola in vocGlove.keys():
      parola_vec=vocGlove[parola]
      compressed=tf.Variable(np.array([parola_vec]).reshape(50,-1), dtype=tf.float32)

      score1[parola]=math.sqrt(distanzaH(compression,compressed).numpy())
    best1=min(score1.values())
    migliore1=[key for key, value in score1.items() if value == best1]
    parola1=vocGlove[migliore1[0]]
    score2={}
    for parola in vocGlove.keys():
      if parola != migliore1:
        parola_vec=vocGlove[parola]
        compressed=tf.Variable(np.array([parola1,parola_vec]).reshape(50,-1), dtype=tf.float32)
        score2[parola]=math.sqrt(distanzaH(compression,compressed).numpy())
    best2=min(score2.values())
    migliore2=[key for key, value in score2.items() if value == best2]
    parola2=vocGlove[migliore2[0]]
    score3={}
    for parola in vocGlove.keys():
      if parola != migliore1 and parola!=migliore2:
        parola_vec=vocGlove[parola]
        compressed=tf.Variable(np.array([parola1,parola2,parola_vec]).reshape(50,-1), dtype=tf.float32)
        score3[parola]=math.sqrt(distanzaH(compression,compressed).numpy())
    best3=min(score3.values())
    migliore3=[key for key, value in score3.items() if value == best3]
    print("[",migliore1[0].upper(),", ",migliore2[0].upper(),", ",migliore3[0].upper(),"]","d(S,C1)=",round(dc,4),"d(S,CP)=",round(best3,4))




Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, 
[ SISTER ,  DISOBEY ,  SONG ] d(S,C1)= 0.6234 d(S,CP)= 0.6831
  So she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a White Rabbit with pink eyes ran close by her
[ SO ,  DETACH ,  EXCLAMATION ] d(S,C1)= 0.6504 d(S,CP)= 0.7101
 nor did Alice think it so _very_ much out of the way to hear the Rabbit say to itself, 
[ HEAR ,  DISOBEY ,  VERSION ] d(S,C1)= 0.6442 d(S,CP)= 0.6888
 (when she thought it over afterwards, it occurred to her that she ought to have wondered at this, but at the time it all seemed quite natural)
[ WONDERED ,  DISOBEY ,  FARMER ] d(S,C1)= 0.7031 d(S,CP)= 0.6481
 

Using as kernel:
$h(x,y)=\arccos{x\cdot y}$

In [13]:
  K=3 #number of Dirac masses in the compressed distribution
  N=50 #ambient dimension
  J=1000 #how many samples are included: more the better, otherwise there are oscillations in the SGD/Adam convergence
  
  for w in range(num_sent):
    random.seed(10)
    sentence=sentencesAsVec[w]
    num_rand=np.random.rand(N,K)
    den_rand=np.tile(np.linalg.norm(num_rand,axis=0),(N,1))
    quant=tf.Variable(num_rand/den_rand,dtype=tf.float32) #this will be the set of quantization variables
    opt = tf.keras.optimizers.Adam(learning_rate=0.01,beta_1=0.9, beta_2=0.999)# TODO: use lr_schedule for step decay schedule !
    Y=empirical_sampling(sentence)
    #Y,indices=empirical_sampling(sentence)
    loss_quant = lambda: empirical_radon_sobolev_distance_sq_geod(quant,Y)

    nr_iter=200
    loss_val=[]
    opt_val=[]

    for sc in range(nr_iter):
      vq=quant.numpy()
      quant.assign(vq/np.linalg.norm(vq,axis=0))
      step_count = opt.minimize(loss_quant, [quant]).numpy()
      opt_val.append(quant.numpy())
      loss_val.append(loss_quant())
    c1=quant[:,0]/np.linalg.norm(quant[:,0])
    c2=quant[:,1]/np.linalg.norm(quant[:,1])
    c3=quant[:,2]/np.linalg.norm(quant[:,2])
    compression=[c1,c2,c3]

    cdcd=tf.Variable(np.array(compression).reshape(50,-1), dtype=tf.float32)
    dc=math.sqrt(distanzaGeo(sentence,cdcd).numpy())
    print(frasi[w])
    #indici=random.sample(range(N), 3)
    score1={}
    for parola in vocGlove.keys():
      parola_vec=vocGlove[parola]
      compressed=tf.Variable(np.array([parola_vec]).reshape(50,-1), dtype=tf.float32)/np.linalg.norm(parola_vec)
      score1[parola]=math.sqrt(distanzaGeo(compression,compressed).numpy())
    #print(score1)
    best1=min(score1.values())
    migliore1=[key for key, value in score1.items() if value == best1]
    parola1=vocGlove[migliore1[0]]/np.linalg.norm(vocGlove[migliore1[0]])
    score2={}
    for parola in vocGlove.keys():
      if parola != migliore1:
        parola_vec=vocGlove[parola]/np.linalg.norm(vocGlove[parola])

        compressed=tf.Variable(np.array([parola1,parola_vec]).reshape(50,-1), dtype=tf.float32)
        score2[parola]=math.sqrt(distanzaGeo(compression,compressed).numpy())
    best2=min(score2.values())
    migliore2=[key for key, value in score2.items() if value == best2]
    parola2=vocGlove[migliore2[0]]
    score3={}
    for parola in vocGlove.keys():
      if parola != migliore1 and parola!=migliore2:
        parola_vec=vocGlove[parola]/np.linalg.norm(vocGlove[parola])
        compressed=tf.Variable(np.array([parola1,parola2,parola_vec]).reshape(50,-1), dtype=tf.float32)
        score3[parola]=math.sqrt(distanzaGeo(compression,compressed).numpy())
    best3=min(score3.values())
    migliore3=[key for key, value in score3.items() if value == best3]
    print("[",migliore1[0].upper(),", ",migliore2[0].upper(),", ",migliore3[0].upper(),"]","d(S,C1)=",round(dc,4),"d(S,CP)=",round(best3,4))




Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, 
[ HER ,  DISOBEY ,  LORY ] d(S,C1)= 0.6587 d(S,CP)= 0.6744
  So she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a White Rabbit with pink eyes ran close by her
[ GETTING ,  DETACH ,  EXCLAMATION ] d(S,C1)= 0.7013 d(S,CP)= 0.7709
 nor did Alice think it so _very_ much out of the way to hear the Rabbit say to itself, 
[ THINK ,  DISOBEY ,  LORY ] d(S,C1)= 0.726 d(S,CP)= 0.7135
 (when she thought it over afterwards, it occurred to her that she ought to have wondered at this, but at the time it all seemed quite natural)
[ TIME ,  DISOBEY ,  FARMER ] d(S,C1)= 0.7249 d(S,CP)= 0.7448
 but w